# A simple example II

### Installation instruction

It is recommended that you clone the qthought repository to your local machine and then run

in the qthought folder. If you did not pip install qthought, you can use the following quick-fix by uncommenting and adapting to your local file path

In [3]:
# import sys
# import os
# to run the example, set the following path to the folder path of qthought on your machine
# sys.path.append(os.path.abspath('/Users/nuri/qthought/qthought')) 

### Start of the example

This is a detailed explanation of how to write a simple protocol for two parties thought experiment. Prior to reading this, it is recommended to take a look at the PDF description file *simple_examples*.

First, we import the ProjectQ operations needed for the protocol. To initialize the state of the qubit Alice and Bob measure sequentially, we need the Hadamard gate.
We also import *Protocol* and *ProtocolStep* classes to be able to define steps of the protocol; *QuantumSystem* to operate quantum systems of different dimensionality; *Agent* class and all functions from the *copenhagen_theory* module.
We don't import *consistency* rules here, as in this example agents don't have to look at each other's viewpoints and glue them together. For an example where they do, please refer to the Frauchiger-Renner thought experiment protocol.

In [6]:
import qthought.utils as ut
from qthought.protocol import Protocol, ProtocolStep
from qthought.quantumsystem import QuantumSystem
from qthought.agents import Agent
from qthought.interpretations.copenhagen_theory import *
from projectq.ops import H

The first action of the protocol (at time $t=0$) is the initilization of a qubit *s* in a $\frac{1}{\sqrt{2}} (|0> + |1>)$ state. After defining the action, we define the step of the protocol by specifying: domain of action; written description of the action, which will be used for printouts during the run; time of the step; and which action variable being described.

In [7]:
def action1(qsys:QuantumSystem):
    H | qsys['s']
    
step1 = ProtocolStep(domain={'Qubit': 's'},
                     descr = 'Prepare Qubit s by applying H ',
                     time  = 0,
                     action = action1)

In the second action ($t=1$) Alice measures *s* and writes down the result to her memory. Here the *observe* function comes in play: first variable corresponds to the memory (*Alice_memory*), and the second variable - to the system being measured (*alice_qubit*). In the *step2* variable, we again specify the step by its domain - which now includes an object *Alice* of the class *Agent* with 1 memory qubit.

In [8]:
def action2(qsys:QuantumSystem):
    observe(qsys['Alice_memory'],qsys['s'])
    
step2 = ProtocolStep(domain={'Qubit': 's',
                             'AgentMemory(1)': 'Alice'},
                     descr = 'Alice observes s',
                     time  = 1,
                     action = action2)

At $t=2$, Bob makes a measurement of *s*, and writes it down to his memory.

In [9]:
def action3(qsys:QuantumSystem):
    observe(qsys['Bob_memory'],qsys['s'])
    
step3 = ProtocolStep(domain={'Qubit': 's',
                             'AgentMemory(1)': 'Bob'},
                     descr = 'Bob observes s',
                     time  = 2,
                     action = action3)

We glue together all steps of the protocol.

In [10]:
#p = Protocol()
p = step1 + step2 + step3

Now we can print out the full protocol and all systems required to run it.

In [11]:
p

Step 0: Prepare Qubit s by applying H (t:0)
Step 1: Alice observes s(t:1)
Step 2: Bob observes s(t:2)

Requirements: 
------------------------------
Qubit             ['s']
AgentMemory(1)    ['Bob', 'Alice']

We are ready to run, and see the evolution of the joint state of systems *s*, *Alice_memory* and *Bob_memory*. In the first line we initialize the quantum system of the setting according to the requirements of the protocol.

In [12]:
qsys = QuantumSystem(p.get_requirements())
p.run(qsys)

(Note: This is the (slow) Python simulator.)
Require Qubit s
Require AgentMemory(1) Bob
Require AgentMemory(1) Alice
0 Prepare Qubit s by applying H  t:0
State:
0.71|000> + 0.71|001>
1 Alice observes s t:1
State:
0.71|000> + 0.71|101>
2 Bob observes s t:2
State:
0.71|000> + 0.71|111>


We can also check the inferences Alice and Bob can make about each other's outcomes. First, we define an inference table of Alice reasoning about Bob. Here we use *forward_inference*, as Alice is reasoning about a later step.

In [13]:
forward_inference(p, 'Alice_memory', 1, 'Bob_memory', 2)

(Note: This is the (slow) Python simulator.)


In:(Alice_memory:t1)  |  Out: (Bob_memory:t2)
----------------------------------------------------
           0          |        [0]
           1          |        [1]

Bob is reasoning about an earlier step, so here we use *backward_inference*.

In [14]:
backward_inference(p, 'Alice_memory', 1, 'Bob_memory', 2)

(Note: This is the (slow) Python simulator.)


In:(Bob_memory:t2)    |  Out: (Alice_memory:t1)
------------------------------------------------------
           0          |        [0]
           1          |        [1]

We see that in our model Alice's and Bob's memories are consistently correlated with each other.